In [1]:
from trainer import Trainer
from TheCar import Car
from time import sleep
from carbase import CarBase
from track import Track, CControlPoint
from utils.Vector2 import Vector2D
from PathPoint import PathP
from Sensors import Sensors
import numpy as np
import random
import math
from collections import deque

In [2]:
# трек
r = Track()
data = [[77,131,18,False],
        [370,56,18,False],
        [711,101,18,False],
        [741,366,18,False],
        [385,368,18,False],
        [69,309,18,False],
        [16,213,18,False],
        [65,138,18,False]]
ind = 0
for d in data:
    r.ControlPoints.append(CControlPoint(Vector2D(d[0],d[1]),d[2]))
    r.ControlPoints[ind].Temp = d[3]
    ind += 1

with open("points.txt", "r") as ins:
    for line in ins:
        data = line.split() 
        p = PathP()
        p.Point.X = float(data[0])
        p.Point.Y = float(data[1])
        p.Width = float(data[2])
        r.Points.append(p)
        
r.Prepare()

#draw Car
from time import sleep
from ipycanvas import Canvas, hold_canvas

canvas = Canvas(width=1000, height=400)
display(canvas)

def draw_cars(car, n):
    with hold_canvas(canvas):
        # Clear the old animation step
        canvas.clear()
        for p in r.Points:  
            canvas.stroke_style = 'blue'
            canvas.stroke_line(p.LeftWall.From.X,p.LeftWall.From.Y,p.LeftWall.To.X,p.LeftWall.To.Y)
            canvas.stroke_line(p.RightWall.From.X,p.RightWall.From.Y,p.RightWall.To.X,p.RightWall.To.Y)
        canvas.fill_style = 'red'
        if car.Breaks:
            canvas.fill_style = 'orange'
        canvas.fill_rect(car.Pos.X, car.Pos.Y, 5, 5)
        #canvas.fill_style = 'white'
        #canvas.fill_rect(cars[2].Pos.X, cars[2].Pos.X, 5, 5)
            
        #canvas.fill_style = 'green'
        #canvas.fill_rect(cars[4].Pos.X, cars[4].Pos.Y, 5, 5)
        #canvas.fill_style = 'black'
        #canvas.fill_rect(cars[9].Pos.X, cars[9].Pos.Y, 5, 5)
            
        canvas.fill_style = 'red'
        canvas.font = '14px serif'
        canvas.fill_text(str(car.Velocity), 0, 10, max_width=None)
        canvas.fill_text(n, 0, 25, max_width=None)
        canvas.fill_text(car.GetPassLength(), 0, 40, max_width=None)
        canvas

        canvas.sleep(10)
        
Tr = Trainer()
Tr.Track = r
Tr.Population = 10
#Tr.GeneticNN.ActivationFunctionType = TanHFunction
generationCount = 2
for i in range(generationCount):
        Tr.BuildFirstPopulation()
        Tr.BuildNextGeneration()

        while (True):
            if (not Tr.IsAlive()):
                if (Tr.Generation >= generationCount):
                    break
                Tr.BuildNextGeneration()
            draw_cars(Tr.find_leader(), i)
            # если лидер прошёл более 10 000 метров, то сохранить его веса в файл
            # Tr.GeneticNN.NNs[Tr.find_leader().Index].SaveWeights("net.pt")
            dt = 0.25
            # update cars
            Tr.Update(dt)

Canvas(height=400, width=1000)